<a href="https://colab.research.google.com/github/Manivannan56/MedMNIST-Classifictaion/blob/main/MedMNIST_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=bf9481c7a5e2cfc51b65c875300b0e623710d415d6abe1892664960876958f5c
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from  torch.utils.data import DataLoader,TensorDataset
from sklearn.metrics import classification_report

import torchvision.transforms as transforms
import torchvision.models as models
import medmnist
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


In [3]:
from medmnist import OrganAMNIST
from medmnist import OrganCMNIST
from medmnist import OrganSMNIST
from medmnist import INFO

In [4]:
data_1='organamnist'
data_2='organcmnist'
data_3='organsmnist'

def get_details(data_flag):
  info=INFO[data_flag]
  task=info['task']
  n_channels=info['n_channels']
  n_classes=len(info['label'])



  return info,task,n_channels,n_classes




In [5]:
info_1,task_1,n_channels_1,n_classes_1=get_details(data_1)
info_2,task_2,n_channels_2,n_classes_2=get_details(data_2)
info_3,task_3,n_channels_3,n_classes_3=get_details(data_3)

Data_class1=getattr(medmnist,info_1['python_class'])
Data_class2=getattr(medmnist,info_2['python_class'])
Data_class3=getattr(medmnist,info_3['python_class'])



In [6]:
Data_transformation=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5],std=[0.5])
])

download=True
batch_size=64
num_epochs=50

In [7]:
from posixpath import split
train_dataset_1=Data_class1(split='train',transform=Data_transformation,download=download)
train_dataset_2=Data_class2(split='train',transform=Data_transformation,download=download)
train_dataset_3=Data_class3(split='train',transform=Data_transformation,download=download)






100%|██████████| 38247708/38247708 [00:02<00:00, 15414690.14it/s]


100%|██████████| 15526411/15526411 [00:01<00:00, 10918531.25it/s]


100%|██████████| 16528359/16528359 [00:01<00:00, 11711627.11it/s]


In [8]:
def relabel_dataset(labels):
  for i in labels:
    i+=11
  return labels

In [9]:
labels_2=relabel_dataset(train_dataset_2.labels)
labels_3=relabel_dataset(train_dataset_3.labels)

In [10]:
images_1=np.array(train_dataset_1.imgs)
labels_1=np.array(train_dataset_1.labels)
images_2=np.array(train_dataset_2.imgs)
labels_2=np.array(labels_2)
images_3=np.array(train_dataset_3.imgs)
labels_3=np.array(labels_3)

images=np.concatenate((images_1,images_2,images_3),axis=0)
labels=np.concatenate((labels_1,labels_2,labels_3),axis=0)

len(images),len(labels)

(61468, 61468)

In [11]:
labels[50000]

array([15], dtype=uint8)

In [12]:
images=torch.from_numpy(images)

images=images.unsqueeze(1).float()
labels=labels.squeeze()


In [13]:
labels.shape

(61468,)

In [14]:
labels[50000]

15

In [15]:

X=torch.tensor(images,dtype=torch.float32)
Y=torch.tensor(labels,dtype=torch.long)

<ipython-input-15-d07992d93ea8>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X=torch.tensor(images,dtype=torch.float32)


In [16]:
X_train,X_val,Y_train,Y_val=train_test_split(X,Y,test_size=0.2,random_state=42)

In [17]:
training_data=TensorDataset(X_train,Y_train)
validation_data=TensorDataset(X_val,Y_val)

train_loader=DataLoader(dataset=training_data,batch_size=batch_size,shuffle=True)
val_loader=DataLoader(dataset=validation_data,batch_size=batch_size,shuffle=False)



In [18]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [19]:
device

'cuda'

In [20]:
class Modified_Resnet(nn.Module):
  def __init__(self,n_classes=33):
    super(Modified_Resnet,self). __init__()

    self.resnet=models.resnet18(pretrained=True)

    self.resnet.conv1=nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1)

    self.resnet.maxpool=nn.Identity()

    num_filters=self.resnet.fc.in_features
    self.resnet.fc= nn.Linear(num_filters,n_classes)

  def forward(self,x):
    return self.resnet(x)


In [21]:
model=Modified_Resnet(n_classes=33)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 55.8MB/s]


In [22]:
model.to(device)

Modified_Resnet(
  (resnet): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): Identity()
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): 

In [23]:
criterion= nn.CrossEntropyLoss()
optimizer= optim.Adam(model.parameters(),lr=0.001)

In [25]:
for epoch in range(num_epochs):
  model.train()
  train_loss=0.0
  all_labels=[]
  all_predictions=[]

  for inputs,labels in train_loader:


     inputs,labels=inputs.to(device),labels.to(device)
     labels = labels.long()

     optimizer.zero_grad()
     outputs=model(inputs)
     loss=criterion(outputs,labels)
     loss.backward()
     optimizer.step()

     train_loss+=loss.item()*inputs.size(0)
     _,predicted=torch.max(outputs.data,1)
     all_labels.extend(labels.cpu().numpy())
     all_predictions.extend(predicted.cpu().numpy())
     train_report=classification_report(all_labels,all_predictions,zero_division=0)
  train_loss/=len(train_loader.dataset)

  model.eval()
  val_loss=0.0
  all_labels_val = []
  all_predictions_val = []
  for inputs,labels in val_loader:

    inputs,labels=inputs.to(device),labels.to(device)
    labels = labels.long()


    optimizer.zero_grad()
    outputs=model(inputs)
    loss=criterion(outputs,labels)
    val_loss+=loss.item()*inputs.size(0)
    _,predicted=torch.max(outputs.data,1)
    all_labels_val.extend(labels.cpu().numpy())
    all_predictions_val.extend(predicted.cpu().numpy())
    val_report=classification_report(all_labels_val,all_predictions_val,zero_division=0)

  val_loss/=len(val_loader.dataset)

  print(f'Epoch {epoch+1}/{num_epochs},'
          f'Training Loss: {train_loss:.4f},'
          f'Validation Loss: {val_loss:.4f}')




Epoch 1/50,Training Loss: 0.2188,Validation Loss: 0.2195
Epoch 2/50,Training Loss: 0.1366,Validation Loss: 0.1797
Epoch 3/50,Training Loss: 0.0932,Validation Loss: 0.1792
Epoch 4/50,Training Loss: 0.0838,Validation Loss: 0.1003
Epoch 5/50,Training Loss: 0.0636,Validation Loss: 0.1124
Epoch 6/50,Training Loss: 0.0523,Validation Loss: 0.1065
Epoch 7/50,Training Loss: 0.0478,Validation Loss: 0.1607
Epoch 8/50,Training Loss: 0.0440,Validation Loss: 0.0950
Epoch 9/50,Training Loss: 0.0378,Validation Loss: 0.1219
Epoch 10/50,Training Loss: 0.0354,Validation Loss: 0.0950
Epoch 11/50,Training Loss: 0.0277,Validation Loss: 0.0845
Epoch 12/50,Training Loss: 0.0319,Validation Loss: 0.0994
Epoch 13/50,Training Loss: 0.0227,Validation Loss: 0.0950
Epoch 14/50,Training Loss: 0.0232,Validation Loss: 0.1035
Epoch 15/50,Training Loss: 0.0271,Validation Loss: 0.1003
Epoch 16/50,Training Loss: 0.0177,Validation Loss: 0.0997
Epoch 17/50,Training Loss: 0.0211,Validation Loss: 0.0874
Epoch 18/50,Training Lo

In [28]:
print(val_report)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       368
           1       0.99      1.00      0.99       313
           2       0.98      0.99      0.98       267
           3       1.00      1.00      1.00       296
           4       0.99      0.99      0.99       743
           5       0.98      0.99      0.99       765
           6       1.00      1.00      1.00      1244
           7       1.00      1.00      1.00       761
           8       1.00      1.00      1.00       794
           9       0.96      0.99      0.98       603
          10       0.98      1.00      0.99       717
          11       0.99      0.98      0.99       454
          12       0.72      0.83      0.77       240
          13       0.83      0.69      0.75       232
          14       0.99      1.00      0.99       255
          15       0.94      0.93      0.94       451
          16       0.94      0.93      0.94       451
          17       1.00    

In [29]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [34]:
torch.save(model,'/content/drive/MyDrive/model_entire_1.pth')

In [35]:
model_test=(torch.load('/content/drive/MyDrive/model_entire_1.pth'))
model_test.eval()

Modified_Resnet(
  (resnet): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): Identity()
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): 

In [36]:
from sklearn.metrics import classification_report
import numpy as np

true_labels = []
predictions = []

with torch.no_grad():  # No gradients needed
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels=labels.to(device)
        outputs = model_test(inputs)
        _, predicted = torch.max(outputs, 1)

        # Collect true labels and predictions
        true_labels.extend(labels.cpu().numpy())
        predictions.extend(predicted.cpu().numpy())

print(classification_report(true_labels, predictions))



              precision    recall  f1-score   support

           0       0.99      1.00      0.99       368
           1       0.99      1.00      0.99       313
           2       0.98      0.99      0.98       267
           3       1.00      1.00      1.00       296
           4       0.99      0.99      0.99       743
           5       0.98      0.99      0.99       765
           6       1.00      1.00      1.00      1244
           7       1.00      1.00      1.00       761
           8       1.00      1.00      1.00       794
           9       0.96      0.99      0.98       603
          10       0.98      1.00      0.99       717
          11       0.99      0.98      0.99       454
          12       0.72      0.83      0.77       240
          13       0.83      0.69      0.75       232
          14       0.99      1.00      0.99       255
          15       0.94      0.93      0.94       451
          16       0.94      0.93      0.94       451
          17       1.00    